In [35]:
# License: BSD
# Author: Sasank Chilamkurthy+Zhang Liangjun
#verson 0.1
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import scipy.io as scio
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import time
import copy
import os
import torch.utils.data as data
import random
import shutil
import glob

#文件路径
#filepath = '/home/gcc/dataset/test-xyz_depth.mat'


In [36]:
#input:mat file
#output: tensor
def mat_read(filepath):
    dataFile = filepath
    data = scio.loadmat(dataFile)
    #读取mat里的depth数据
    depth = data['depth']
    #归一化
    depth_scale=depth[depth>0]
    avrg=np.mean(depth_scale)
    var=np.std(depth_scale)
    index=depth==0
    depth[index]=avrg
    depth_scale=(depth-avrg)/var
    ##TODO
    #where_are_nan = np.isnan(depth)
    #depth[where_are_nan] = 0
    #变成三通道
    depth3=[]
    for i in range(3):
        depth3.append(depth_scale)
    depth3=np.array(depth3)
    #create tensor from numpy.ndarray
    depth=torch.from_numpy(depth3)
    depth=depth.type(torch.FloatTensor)
    return depth

#把数据文件分成训练集和测试集
def generate_datasets(data_dir, dst_dir, train_ratio=0.7):
    train_dir = os.path.join(dst_dir,'train')
    val_dir = os.path.join(dst_dir,'val')
    if not os.path.exists(dst_dir):
        os.mkdir(dst_dir)
        os.mkdir(train_dir)
        os.mkdir(val_dir)
    for root, dirnames, _ in os.walk(data_dir):
        for dirname in dirnames:
            #因为需要的数据在更下一层的depth文件里，所以加了几句，后边带俩#的为改过的部分，
            #如果不需要做depth改过来就行了
            #depth_dirname=os.path.join(dirname,'depth')##
            subdirname_train = os.path.join(train_dir, dirname)
            subdirname_val = os.path.join(val_dir, dirname)
            if not os.path.exists(subdirname_train):
                os.mkdir(subdirname_train)
            if not os.path.exists(subdirname_val):
                os.mkdir(subdirname_val)                                        
            #dname = os.path.join(root, depth_dirname)##depth_dirname原本是dirname
            dname = os.path.join(root, dirname)
            names = glob.glob(dname+r'/*.mat')  
            random.shuffle(names)
            names_len = len(names)
            train_names = names[:int(names_len*train_ratio)]
            val_names = names[int(names_len*train_ratio)+1:]
            for f in train_names:
                fname = os.path.split(f)[-1]
                train_dname = os.path.join(subdirname_train, fname)
                shutil.copyfile(f, train_dname)
            for f in val_names:
                fname = os.path.split(f)[-1]
                val_dname = os.path.join(subdirname_val, fname)
                shutil.copyfile(f, val_dname)
            print ('copy {} done'.format(dname))
        
data_dir = '/home/sjtu/gcj/data/depth_data/depthmat2'
dst_dir = '/home/sjtu/gcj/data/depth_data/depthmat_new'
#generate_datasets(data_dir, dst_dir)
        

In [37]:
EXTENSIONS = ['.mat']
def is_mat_file(filename):
    return any(filename.endswith(extension) for extension in EXTENSIONS)

#类名
def find_classes(dir):
    classes = [d for d in os.listdir(dir) if os.path.isdir(os.path.join(dir,d))]
    classes.sort()
    class_to_idx = {classes[i]:i for i in range(len(classes))}
    return classes,class_to_idx

#input: dir+train(or val)+class
#output: 数据文件的集合
def make_dataset(dir,phase,class_to_idx):
    datas = []
    dir = os.path.join(dir,phase)
    for target in os.listdir(dir):
        d = os.path.join(dir,target)
        if not os.path.isdir(d):
            continue
        
        for root, _, fnames in sorted(os.walk(d)):
            for fname in fnames:
                if is_mat_file(fname):
                    path = os.path.join(root,fname)
                    item = (path,class_to_idx[target])
                    datas.append(item)
    return datas



In [38]:
class ViewpointDataset(data.Dataset):
    def __init__(self, root, transform = None, phase = None):
        dir = os.path.join(root, phase)
        classes, class_to_idx = find_classes(dir)
        datas = make_dataset(root,phase, class_to_idx)
        if len(datas) == 0:
            raise(RuntimeError("Found 0 images in subfolders of: " + root + "\n"
                               "Supported extensions are: " + ",".join(EXTENSIONS)))
        self.root = root
        self.datas = datas
        self.phase = phase
        self.classes = classes
        #todo
        self.width = 480
        self.height = 640
        self.suffix = '.mat'
        self.transform = transform
        
    #深度矩阵转成tensor  
    def __getitem__(self, idx):
        mat_path, label = self.datas[idx]
        #preprocess
        depth_tensor = mat_read(mat_path)            
        
        #numpy转成tensor  (不用了，matread里边转过tensor了)
        #depth_tensor = torch.from_numpy(depth)
        return depth_tensor, label
    
    def __len__(self):
        return len(self.datas)

In [39]:
#建立数据集
dsets = {x: ViewpointDataset(dst_dir,phase=x) for x in ['train', 'val']}
dset_loaders = {x:torch.utils.data.DataLoader(dsets[x],batch_size=24,shuffle=True, num_workers=8) for x in ['train', 'val']}
dset_sizes = {x: len(dsets[x]) for x in ['train', 'val']}
dset_classes = dsets['val'].classes
print(dset_sizes)
print(len(dset_classes))
use_gpu = torch.cuda.is_available()
#use_gpu = False

{'train': 24864, 'val': 10619}
37


In [40]:
#设置梯度更新方式
def optim_scheduler_ft(model, epoch, init_lr=0.001, lr_decay_epoch=7):
    lr = init_lr * (0.1**(epoch // lr_decay_epoch))

    if epoch % lr_decay_epoch == 0:
        print('LR is set to {}'.format(lr))

    #optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    return optimizer

In [41]:
#建立模型
model = models.resnet34(pretrained=False)
#model还可以是各种卷积网络结构，到时候调整
#model = torch.load("model_epoch25.pkl")
print (model)
num_ftrs = model.fc.in_features
classes = len(dset_classes)
model.fc = nn.Linear(2048, classes)
if use_gpu:
    model = model.cuda()

criterion = nn.CrossEntropyLoss()

#删除最后一层
#new_classifier = nn.Sequential(*list(model.children())[:-1])
#print (new_classifier)

#测试网络
#a=mat_read('/home/gcc/viewpoint/off_1_90_0/depth/1_depth.mat')
#c=torch.FloatTensor(1,3,480,640)#需要写成3dtensor 1代表batch
#c[0]=a
#b=Variable(c.cuda())
#print(type(b))
#outputs = new_classifier(b)


ResNet (
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
  (relu): ReLU (inplace)
  (maxpool): MaxPool2d (size=(3, 3), stride=(2, 2), padding=(1, 1), dilation=(1, 1))
  (layer1): Sequential (
    (0): BasicBlock (
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
      (relu): ReLU (inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
    )
    (1): BasicBlock (
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
      (relu): ReLU (inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d

In [42]:
#print outputs

In [43]:
##定义模型如何训练
def train_model(model, criterion, optim_scheduler, num_epochs=25):
    since = time.time()

    best_model = model
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                optimizer = optim_scheduler(model, epoch)

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for data in dset_loaders[phase]:
                # get the inputs
                inputs, labels = data
                #print(type(inputs))
                # wrap them in Variable
                if use_gpu and phase=='train':
                    inputs, labels = Variable(inputs.cuda()), Variable(labels.cuda())
                elif use_gpu and phase=='val':
                    inputs, labels = Variable(inputs.cuda(),volatile=True), Variable(labels.cuda(),volatile=True)
                else:
                    inputs, labels = Variable(inputs), Variable(labels)
                #print(type(inputs))
                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                outputs = model(inputs)
                _, preds = torch.max(outputs.data, 1)
                loss = criterion(outputs, labels)

                # backward + optimize only if in training phase
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

                # statistics
                running_loss += loss.data[0]
                running_corrects += torch.sum(preds== labels.data)
                #print(running_loss)

            epoch_loss = running_loss / dset_sizes[phase]
            epoch_acc = float(running_corrects) / float(dset_sizes[phase])

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model = copy.deepcopy(model)


    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))
    return best_model

In [44]:
#训练
model = train_model(model, criterion, optim_scheduler_ft, num_epochs=25)

Epoch 0/24
----------
LR is set to 0.001
train Loss: 0.0301 Acc: 0.7734
val Loss: 0.0061 Acc: 0.9516
Epoch 1/24
----------
train Loss: 0.0053 Acc: 0.9571
val Loss: 0.0018 Acc: 0.9847
Epoch 2/24
----------
train Loss: 0.0025 Acc: 0.9809
val Loss: 0.0015 Acc: 0.9885
Epoch 3/24
----------
train Loss: 0.0016 Acc: 0.9879
val Loss: 0.0010 Acc: 0.9914
Epoch 4/24
----------
train Loss: 0.0008 Acc: 0.9937
val Loss: 0.0002 Acc: 0.9982
Epoch 5/24
----------
train Loss: 0.0008 Acc: 0.9939
val Loss: 0.0015 Acc: 0.9866
Epoch 6/24
----------
train Loss: 0.0006 Acc: 0.9951
val Loss: 0.0008 Acc: 0.9937
Epoch 7/24
----------
LR is set to 0.0001
train Loss: 0.0001 Acc: 0.9996
val Loss: 0.0000 Acc: 0.9998
Epoch 8/24
----------
train Loss: 0.0000 Acc: 0.9999
val Loss: 0.0000 Acc: 1.0000
Epoch 9/24
----------
train Loss: 0.0000 Acc: 1.0000
val Loss: 0.0000 Acc: 1.0000
Epoch 10/24
----------
train Loss: 0.0000 Acc: 0.9999
val Loss: 0.0000 Acc: 0.9999
Epoch 11/24
----------
train Loss: 0.0000 Acc: 1.0000
val 

In [45]:
##在测试集中测试
def visualize_model(model, num_images=5):
    for i, data in enumerate(dset_loaders['val']):
        inputs, labels = data
        if use_gpu:
            inputs, labels = Variable(inputs.cuda()), Variable(labels.cuda())
        else:
            inputs, labels = Variable(inputs), Variable(labels)

        
        outputs = model(inputs)
        _, preds = torch.max(outputs.data, 1)
        
        preds = preds.cpu().numpy()
        print dset_classes[preds[0][0]]

        if i == num_images - 1:
            break

In [13]:
#单个图测试
def predict_depth_mat(model,dep_path):
    inputs = mat_read(dep_path)
    if use_gpu:
        inputs = Variable(inputs.cuda())
    outputs = model(inputs)
    _,preds = torch.max(outputs.data, 1)
    preds = preds.cpu().numpy()
    print dset_classes[preds[0][0]]
#等到知道真实深度图是什么类型的数据和怎么处理后再写
def predict_depthmap(model,image):
    pass
#filename = 'mat文件地址'
#predict_depth_mat(model,filename)

In [ ]:
#保存模型
#torch.save(model, 'model_epoch50.pkl')